# installs and imports

In [1]:
# installs and imports 

!pip install selenium  
!apt-get update
!apt-get upgrade
!apt install chromium-chromedriver
!apt-get update
!apt-get upgrade
!pip install import-ipynb
!apt-get update
!apt-get upgrade

# drive mounting, path appending
mnt = '/content/gdrive'
# ROOT = 'path/to/Austin_Drinks'
ROOT = 'gdrive/MyDrive/Austin_Drinks'
from google.colab import drive
drive.mount(mnt)
import sys
sys.path.append(ROOT)
sys.path.append(ROOT + '/Data')
sys.path.append(ROOT + '/Data/search_result_links')
sys.path.append(ROOT + '/Data/json_by_city')
sys.path.append(ROOT + '/Data/csv_by_city')
print(sys.path)

import import_ipynb
from datahandler import datacollector as dc

import time
import os
import re

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [2]:
SITES =   { 
            'yelp' :  {
                        # base url for yelp
                        'url':'https://www.yelp.com',

                        # in Method get_search_result_links            
                        # search_results_tags 
                        'search_results_tags' : 'h4',           
                        #'search_results_tags_class' : 'css-uvzfg9',

                        # get_search_results_ext_tag_and_attribute
                        'get_search_results_ext_tag_and_attribute' : {'class' : 'css-uvzfg9'},
                       
                        # next page tag in Method get_search_result_links
                        #'next_page_tag' : 'a', anchor tags contain the href attribute
                        'next_page_tag_attr_and_def' : {'class' : 'next-link navigation-button__09f24__m9qRz css-1pxws0l'},
                        'ad_characters' : '/adredir?',
                        #'next_page_tag_attr' : 'class',
                        #'next_page_tag_attr_def' : 'next-link navigation-button__09f24__m9qRz css-1pxws0l',
                        'remove_unwanted_characters_in_url' : '\?osq=+[A-Za-z]+',


                        # in get_yelp_establishment_information: 
                        'establishment_name_tag' : 'h1',
                        'establishment_name_tag_attr_and_def' : {'class' : 'css-1x9iesk'},                      
                       
                        # attributes of button to reveal accommodations and unaccommodations
                        'amenities_header_text' : 'Amenities and More',
                        'amenities_reveal_button_text' : 'Attribute',
                        'amenities_reveal_button_hider_attr' : 'aria-controls',
                        'amenities_reveal_button_tag_attr' : 'class',
                        'amenities_reveal_button_tag_attr_def' : ' css-zbyz55',
                       
                        # status text to determine if the establishment is permanently closed
                        'status_text' : 'this location has closed',
                       
                       
                        # tags containing coordinates information
                        'location_tag' : 'div', 
                        'location_tag_attr_and_def' : {'class' : ' container__09f24__fZQnf border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG'},
                        'location_string_start' : '.png%7C',
                        'location_string_end' : '&',
                        'location_string_delimiter' : '%2C',
                       

                        # tag information to get businesses url
                        'business_url_tag' : 'div',
                        'business_url_tag_attr_and_def' : {'class' : ' css-1vhakgw border--top__09f24__exYYb border-color--default__09f24__NPAKY'},
                        #'business_url_tag_attr' : 'class',
                        #'business_url_tag_attr_def' : ' css-1vhakgw border--top__09f24__exYYb border-color--default__09f24__NPAKY',
                       
                        # tag information to get establishment's tags
                        'est_tags_tag' : 'span',
                        'est_tags_tag_attr_and_def' : {'class' : ' display--inline__09f24__c6N_k margin-r1__09f24__rN_ga border-color--default__09f24__NPAKY'},
                        #'est_tags_tag_attr' : 'class',
                        #'est_tags_tag_attr_def' : ' display--inline__09f24__c6N_k margin-r1__09f24__rN_ga border-color--default__09f24__NPAKY',
                       
                        # hours of operation table tag information
                        'hoO_tag' : 'table',
                        'hoO_tag_attr_and_def' : {'class' : ' hours-table__09f24__KR8wh table__09f24__J2OBP table--simple__09f24__vy16f'},
                        #'hoO_tag_attr' : 'class',
                        #'hoO_tag_attr_def' : ' hours-table__09f24__KR8wh table__09f24__J2OBP table--simple__09f24__vy16f',
                        'hoO_row_tag_attr_and_def' : {'class' : ' table-row__09f24__YAU9e'},
                        #'hoO_row_tag_attr' : 'class',
                        #'hoO_row_tag_attr_def' : ' table-row__09f24__YAU9e',
                       
                        # amenities and anti-amenities tag information
                        'amenities_tag' : 'span', 
                        #'amenities_tag_attr' : 'class', 
                        #'amenities_tag_attr_def' : ' css-1h7ysrc', 
                        'amenities_tag_attr_and_def' : {'class' : ' css-1h7ysrc'},
                        'anti_amenities_tag' : 'span', 
                        #'anti_amenities_tag_attr' : 'class', 
                        #'anti_amenities_tag_attr_def' : ' css-1ccncw',
                        'anti_amenities_tag_attr_and_def' : {'class' : ' css-1ccncw'},
                        
                      }
           }

# pass the SITES dictionary to the package and setup constants for the package
dc.set_constants(SITES, ROOT) 

root dir: gdrive/MyDrive/Austin_Drinks
{'yelp': {'url': 'https://www.yelp.com', 'search_results_tags': 'h4', 'get_search_results_ext_tag_and_attribute': {'class': 'css-uvzfg9'}, 'next_page_tag_attr_and_def': {'class': 'next-link navigation-button__09f24__m9qRz css-1pxws0l'}, 'ad_characters': '/adredir?', 'remove_unwanted_characters_in_url': '\\?osq=+[A-Za-z]+', 'establishment_name_tag': 'h1', 'establishment_name_tag_attr_and_def': {'class': 'css-1x9iesk'}, 'amenities_header_text': 'Amenities and More', 'amenities_reveal_button_text': 'Attribute', 'amenities_reveal_button_hider_attr': 'aria-controls', 'amenities_reveal_button_tag_attr': 'class', 'amenities_reveal_button_tag_attr_def': ' css-zbyz55', 'status_text': 'this location has closed', 'location_tag': 'div', 'location_tag_attr_and_def': {'class': ' container__09f24__fZQnf border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG'}, 'location_string_start': '.png%7C', 'location_string_end': '&', 'location_string_delimiter'

# experiments and adjustments


In [ ]:
# used to flip longitude and latitude
'''
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    tmp = est['longitude']
    est['longitude'] = est['latitude']
    est['latitude'] = tmp
  dc.save_city_json(file, city_est)
  time.sleep(3)
'''

In [ ]:
# used to convert integers and decimals to string
# scratch that, this has been resolved on swift end.
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    tmp = est['rating_value']
    est['rating_value'] = str(tmp)
    tmp = est['review_count']
    est['review_count'] = str(tmp)
  dc.save_city_json(file, city_est)
  time.sleep(3)

In [ ]:
# used to fix reviewRating values
file = 'Marble Falls.json'
city_est = dc.load_city_json(file)
for est in city_est:
  for review in est['reviews']:
    tmp = review['reviewRating']
    review['reviewRating'] = str(tmp)
dc.save_city_json(file, city_est)

In [ ]:
# make review counts in Marble Falls strings.
file = 'Marble Falls.json'
city_est = dc.load_city_json(file)
for est in city_est:
  tmp = est['review_count']
  est['review_count'] = str(tmp)
dc.save_city_json(file, city_est)

In [ ]:
# change file names for app.
'''
import os
import re

files = os.listdir(ROOT + '/Data/json_by_city')
for file in files:
  newname = re.sub('_', '', file)
  newname = re.sub('TX', '', file)
  os.rename(ROOT + '/Data/json_by_city/' + file, ROOT + '/Data/json_by_city/' + newname)
'''

In [ ]:
# see all beer tags
city = 'austin_TX.json'
city_files = dc.get_city_json_filenames()
tags = set()
city_est = dc.load_city_json(city)
print(city)
for est in city_est:
  for tag in est['tags']:
    tags.add(tag)
print(tags)

In [ ]:
# see all days of the week strings

days = set()
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    for days_dict in est['hours'].keys():
      days.add(days_dict)
  dc.save_city_json(file, city_est)
  time.sleep(3)

In [3]:
# reformat dictionaries to include ids towards the top.

def reformat_json(data):
  '''
  # reformat json data to a preferred readable format
  '''
  new_data = {}
  new_data['name'] = data['name']
  new_data['id'] = data['id']
  new_data['open_for_business'] = data['open_for_business']
  new_data['tags'] = data['tags']
  new_data['website'] = data['website']
  new_data['longitude'] = data['longitude']
  new_data['latitude'] = data['latitude']
  new_data['address'] = data['address']
  new_data['postal_code'] = data['postal_code']
  new_data['city'] = data['city']
  new_data['state'] = data['state']
  new_data['telephone'] = data['telephone']
  new_data['price'] = data['price']
  new_data['hours'] = data['hours']
  new_data['accommodations'] = data['accommodations']
  new_data['unaccommodations'] = data['unaccommodations']
  new_data['rating_value'] = data['rating_value']
  new_data['review_count'] = data['review_count']
#  new_data['image'] = data['image']
  new_data['reviews'] = data['reviews']
  return new_data

In [8]:
# add IDs to each establishment
# first add all names to a set
estNames = set()
allEstDetail = {}
estCounter = 0


city_files = dc.get_city_json_filenames()
for file in city_files:
  city_est = dc.load_city_json(file)
  estDetail = {}

  for est in city_est:
    estNames.add(est['name'])
    if est['name'] not in allEstDetail:
      allEstDetail[est['name']] = {'cityCount': 1}
      estCounter += 1
    else:
      allEstDetail[est['name']]['cityCount'] += 1
  time.sleep(1)





In [9]:
print(estCounter)

664


In [11]:
for key, val in allEstDetail.items():
  print(key + '\t\t\t' + str(val['cityCount']))

Save The World Brewing			1
Bear King Brewing Co.			1
Double Horn Brewing			1
Fiesta Winery			1
River City Grille			2
Brass Hall			1
Iron Wolf Ranch and Distillery			3
Heart of Texas Wine Tours			4
Round Mountain Distilling			2
Spicewood Vineyards			3
Texas Tipsy Tours			11
Taste of the Hills			9
Texan Wine and Spirit Tours			5
ThunderDome Party Bus			9
Discover Texas Wine Tours			8
Austin Nites Party Bus			9
Cool Bus Atx			9
All About Central Texas			7
Perissos Vineyard and Winery			1
Flat Creek Enoteca			1
Horseshoe Bay Resort			1
AJE Transportation			3
Treaty Oak Distilling			3
Dripping Springs Distilling			2
Desert Door			5
Deep Eddy Vodka Tasting Room			4
One Shot Distillery and Brewery			2
Frog Pond Distillery			4
Stinson Distilling			4
Revolution Spirits			2
Crowded Barrel Whiskey			3
Goodnight Loving Vodka Distillery + Tasting Room			2
Andalusia Whiskey			1
Cypress Creek Reserve			4
Bell Springs Winery			3
Sidecar Tasting Room			3
Still Austin Whiskey Co.			3
Hye Rum			2
Moonshi

In [6]:

# assign ids to each establishment
dictIDs = {}
id = 1
for name in estNames:
  dictIDs[name] = id
  print(name + '\t\t\t' + str(id))
  id += 1

Red Horn Coffee House and Brewing Company			1
The Hoppy Monk			2
Mako's On The Creek			3
Mort Subite			4
Celis Brewery			5
Fall Creek Vineyards - Driftwood			6
Fixe Austin's Southern House			7
Esperanza Winery			8
Joe Blue's			9
Augusta Vin Winery			10
Vintage Car Service			11
Joyrides and Detours			12
Dog Haus			13
Blue Lotus Winery			14
Trattoria Lisina			15
Planet Rock Vodka Distillery			16
Holiday Inn Express and Suites Fredericksburg			17
Fawncrest Vineyard			18
C.L. Butaud Wines			19
Bull Creek Brewery			20
The Brewtique			21
Eddie V's Prime Seafood			22
Rentsch Brewery			23
The Beer Museum			24
The Tavern			25
Stinson Distilling			26
Alamo Drafthouse Cinema South Lamar			27
Austin Brewery Tours			28
Pizza Delight			29
Hopdoddy Burger Bar			30
Salt Lick Cellars			31
Growler Express			32
Dancing Bear Cantina			33
Hidden Tavern			34
Treff's Tavern			35
Texas Beer Bus			36
Bombshells Restaurant and Bar			37
ATX Beer Bus			38
Fat Ass Winery Tasting Room			39
Frazier's Long and Low			

In [13]:

# iterate through each file
# add the unique ids to the establishments
for file in city_files:
  city_est = dc.load_city_json(file)
  new_city_est = []
  for est in city_est:
    est['id'] = dictIDs[est['name']]
    new_est = reformat_json(est)
    new_city_est.append(new_est)
  dc.save_city_json(file, new_city_est)
  time.sleep(1)

city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Marble Falls.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Dripping Springs.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Wimberley.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Georgetown.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Austin.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Round Rock.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Antonio.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Cedar Park.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Marcos.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Fredericksburg.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/New Braunfels.json
